In [1]:
import os
import sys
import glob
import pydicom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn import Conv2d as Conv2d
from torch.nn import ConvTranspose2d as ConvTranspose2d
from torch.nn import MaxPool2d as MaxPool2d

from data.mask_functions import *
from data.DataLoader import DataLoader
from data.MyUNet import MyUNet

%matplotlib inline

In [2]:
def dice_(y_true, y_pred, smooth = 1):
    assert(y_true.shape[0] == y_pred.shape[0])
    batchsize = y_true.shape
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = torch.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth)/(torch.sum(y_true_f) + torch.sum(y_pred_f) + smooth)

def show_batch(X, Y):
    assert(X.shape == Y.shape)
    c, h, w = X[0].shape
    fig, ax = plt.subplots(nrows=1, ncols=X.shape[0], sharex = True, sharey=True, figsize=(70,70))
    for i in range(X.shape[0]):
        X_re = X[i].reshape((h, w))
        Y_re = Y[i].reshape((h,w))
        ax[i].imshow(X_re, cmap = plt.cm.bone)
        ax[i].imshow(Y_re, alpha = 0.3, cmap = "Reds")

In [3]:
train_path = 'data/full/train/*/*/*.dcm'
labels_path = 'data/full/train-rle.csv'
test_path = ''

data_obj = DataLoader(train_path, test_path, labels_path, batchsize = 8, create_val = False)
data_obj.load_data()
#data_obj.train_bnum = 0
#X, Y = data_obj.get_next_batch()
#show_batch(X, Y)

In [4]:
dtype = torch.cuda.FloatTensor
device = "cuda"
#if torch.cuda.is_available():
#    device = torch.device("cuda")
#else:
#    device = torch.device("cpu")
#print("device is", device)


In [5]:
model = MyUNet()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0008)
model.cuda()

loss_fn = torch.nn.BCELoss().cuda()

In [6]:
num_epochs = 10

root_path = "model/"

for epoch in range(num_epochs):
    running_loss = 0.0
    data_obj.train_bnum = 0
    data_obj.create_datafolds(85.0, 15.0, 0.0)
    #data_obj.create_datafolds(20.0, 10.0, 0.0)
    num_iterations = int(len(data_obj.train_split) / data_obj.batchsize)
    print(num_iterations)
    for i in range(num_iterations):
        X_batch, Y_batch = data_obj.get_next_batch()
        X_batch, Y_batch = torch.from_numpy(X_batch), torch.from_numpy(Y_batch)
        X_batch, Y_batch = X_batch.to(device = device), Y_batch.to(device = device)
        X_batch, Y_batch = X_batch.type(dtype = dtype), Y_batch.type(dtype = dtype)
        X_batch, Y_batch = X_batch.view((-1, 1, 128, 128)), Y_batch.view((-1, 1, 128, 128))
        
        optimizer.zero_grad()
        
        Y_pred = model(X_batch)        
        
        loss = loss_fn(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%50 == 49:
            print('[%d %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 50))
            running_loss = 0
    path_var = root_path + str(epoch)
    torch.save(model, path_var)

print("finished with the epochs")

1138


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 3.95 GiB total capacity; 3.25 GiB already allocated; 6.31 MiB free; 13.70 MiB cached)